# Detecting null values

Importing libraries

In [25]:
import pandas as pd
import numpy as np
import requests
import json

In [26]:
# No Normalized

main_data = requests.get('https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json')
result = json.loads(main_data.text)

# Normalized

main_data_normalized = pd.json_normalize(result)

main_data_normalized.sample(10)

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
5774,7890-VYYWG,Yes,Male,1,Yes,No,3,No,No phone service,DSL,...,No,No,No,Yes,No,Month-to-month,Yes,Mailed check,36.45,93.7
867,1226-IENZN,No,Male,1,No,No,16,Yes,No,Fiber optic,...,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,78.75,1218.25
5368,7328-OWMOM,No,Female,0,No,Yes,31,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Credit card (automatic),20.10,589.25
4923,6728-CZFEI,No,Female,0,No,No,15,Yes,No,DSL,...,No,No,No,Yes,No,One year,No,Mailed check,56.15,931.9
5632,7691-XVTZH,No,Female,0,Yes,No,68,Yes,Yes,Fiber optic,...,No,Yes,No,Yes,No,Two year,Yes,Bank transfer (automatic),89.95,5974.3
1187,1697-LYYYX,No,Female,0,Yes,Yes,61,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Credit card (automatic),19.10,1143.8
4119,5649-TJHOV,Yes,Male,1,Yes,No,27,No,No phone service,DSL,...,No,No,No,No,Yes,Month-to-month,No,Bank transfer (automatic),36.50,1032
5646,7711-GQBZC,No,Female,0,Yes,Yes,71,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic),24.70,1810.55
2899,4016-BJKTZ,No,Female,0,No,No,25,Yes,Yes,Fiber optic,...,Yes,Yes,Yes,Yes,Yes,Two year,No,Electronic check,108.90,2809.05
5584,7632-YUTXB,No,Female,0,Yes,Yes,25,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,20.05,520.1


In [27]:
main_data_normalized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7267 non-null   object 
 1   Churn                      7267 non-null   object 
 2   customer.gender            7267 non-null   object 
 3   customer.SeniorCitizen     7267 non-null   int64  
 4   customer.Partner           7267 non-null   object 
 5   customer.Dependents        7267 non-null   object 
 6   customer.tenure            7267 non-null   int64  
 7   phone.PhoneService         7267 non-null   object 
 8   phone.MultipleLines        7267 non-null   object 
 9   internet.InternetService   7267 non-null   object 
 10  internet.OnlineSecurity    7267 non-null   object 
 11  internet.OnlineBackup      7267 non-null   object 
 12  internet.DeviceProtection  7267 non-null   object 
 13  internet.TechSupport       7267 non-null   objec

Find null values

In [28]:
null_counts = main_data_normalized.isnull().sum()
print(f'Null values: \n{null_counts[null_counts > 0]}')
empty_strings = (main_data_normalized == '').sum()
print(f'Empty values: \n{empty_strings[empty_strings > 0]}')
rows_with_empty = main_data_normalized[main_data_normalized.eq('').any(axis=1)]
print(f'Rows with empty: \n{rows_with_empty.head()}')

Null values: 
Series([], dtype: int64)
Empty values: 
Churn    224
dtype: int64
Rows with empty: 
     customerID Churn customer.gender  customer.SeniorCitizen  \
30   0047-ZHDTW                Female                       0   
75   0120-YZLQA                  Male                       0   
96   0154-QYHJU                  Male                       0   
98   0162-RZGMZ                Female                       1   
175  0274-VVQOQ                  Male                       1   

    customer.Partner customer.Dependents  customer.tenure phone.PhoneService  \
30                No                  No               11                Yes   
75                No                  No               71                Yes   
96                No                  No               29                Yes   
98                No                  No                5                Yes   
175              Yes                  No               65                Yes   

    phone.MultipleLines intern

In [29]:
main_data_cleaned = main_data_normalized[main_data_normalized['Churn'] != '']


In [30]:
# Elimina filas con cualquier valor nulo
main_data_cleaned = main_data_normalized.dropna()

# Encuentra y elimina filas que tienen al menos un campo vacío ('')
main_data_cleaned = main_data_cleaned[~(main_data_cleaned == '').any(axis=1)]

# Convertir a numérico (forzando errores como NaN para no romper el código)
main_data_cleaned['account.Charges.Total'] = pd.to_numeric(main_data_normalized['account.Charges.Total'], errors='coerce')

# Si aparecen nuevos NaN después de la conversión, los quitamos
main_data_cleaned = main_data_cleaned.dropna(subset=['account.Charges.Total'])

# Reinicia indices
main_data_cleaned = main_data_cleaned.reset_index(drop=True)

print(f'Original lenght: {len(main_data_normalized)}\nDataFrame Cleaned Lemght: {len(main_data_cleaned)}\nRecords eliminated: {len(main_data_normalized) - len(main_data_cleaned)}')

main_data_cleaned.info()

Original lenght: 7267
DataFrame Cleaned Lemght: 7032
Records eliminated: 235
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7032 entries, 0 to 7031
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7032 non-null   object 
 1   Churn                      7032 non-null   object 
 2   customer.gender            7032 non-null   object 
 3   customer.SeniorCitizen     7032 non-null   int64  
 4   customer.Partner           7032 non-null   object 
 5   customer.Dependents        7032 non-null   object 
 6   customer.tenure            7032 non-null   int64  
 7   phone.PhoneService         7032 non-null   object 
 8   phone.MultipleLines        7032 non-null   object 
 9   internet.InternetService   7032 non-null   object 
 10  internet.OnlineSecurity    7032 non-null   object 
 11  internet.OnlineBackup      7032 non-null   object 
 12  internet.DeviceProtection  

In [31]:
main_data_cleaned

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.30
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.40
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,9987-LUTYD,No,Female,0,No,No,13,Yes,No,DSL,...,No,No,Yes,No,No,One year,No,Mailed check,55.15,742.90
7028,9992-RRAMN,Yes,Male,0,Yes,No,22,Yes,Yes,Fiber optic,...,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,85.10,1873.70
7029,9992-UJOEL,No,Male,0,No,No,2,Yes,No,DSL,...,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,50.30,92.75
7030,9993-LHIEB,No,Male,0,Yes,Yes,67,Yes,No,DSL,...,No,Yes,Yes,No,Yes,Two year,No,Mailed check,67.85,4627.65


In [32]:
main_data_cleaned.to_csv('../data/processed/data_without_nulls.csv', index = False, sep = ',')